##                      Scientific Caluculation & Machine Learning
##                      2022-23 Semester 2, AssessmentTask 2
##                      Due time: CNT 23:59, SAT. 5 May 2023
## Subject lecturer: Prof. Hong-qian Sang
## Dept. : School of Artificial Intelligence 

# 导入库
导入了三个Python包

numpy用于数值计算，
scipy用于科学计算
bqplot用于数据可视化。
bqplot中的pyplot模块则被重命名为plt以便更方便地在代码中使用。



In [1]:
%matplotlib notebook
import numpy as np
from scipy import integrate
import bqplot as bq
from bqplot import pyplot as plt

# 函数定义：
## 函数头
定义了一个函数f，该函数采用了三个参数：t、y和args。t:时间，y:状态向量，args是一个元组(arguement)，包含了四个参数：G、m_A、m_B和m_C，分别表示万有引力常数、物体A、B和C的质量。

## 函数体
函数f将状态向量y分成了6个3维向量，分别表示A、B、C三个物体的位置和速度。然后，根据牛顿万有引力定律计算了三个物体之间的引力作用，并计算了每个物体的加速度，最后返回一个包含位置和速度的向量。

## 函数功能

函数f的主要功能是用于求解三体问题的运动方程，根据初始条件和牛顿力学定律计算出三个物体的运动轨迹。具体来说，它计算了每个物体在三个方向上的加速度，然后将加速度和速度合并，返回状态向量的导数，作为ODE求解器的输入，用于求解三体问题的解

In [2]:
def f(t, y, args):
    G, m_A, m_B, m_C = args
    pos_A, pos_B, pos_C, vel_A, vel_B, vel_C = y[:3], y[3:6], y[6:9], y[9:12], y[12:15], y[15:]
    r_AB = np.sqrt(np.sum((pos_A-pos_B)**2))
    r_BC = np.sqrt(np.sum((pos_B-pos_C)**2))
    r_CA = np.sqrt(np.sum((pos_C-pos_A)**2))
    F_A = m_A * m_B * G*(pos_B-pos_A)/r_AB**3 + m_C * m_A * G*(pos_C-pos_A)/r_CA**3
    F_B = m_A * m_B * G*(pos_A-pos_B)/r_AB**3 + m_C * m_B * G*(pos_C-pos_B)/r_BC**3
    F_C = m_A * m_C * G*(pos_A-pos_C)/r_CA**3 + m_C * m_B * G*(pos_B-pos_C)/r_BC**3
    return np.hstack((vel_A, vel_B, vel_C, F_A/m_A, F_B/m_B, F_C/m_C))

# 开始初始化参数

In [3]:
G = 10.
m_A = 1.
m_B = 1.
m_C = 1.

args = (G, m_A, m_B, m_C)

## 参数含义
pos_A、vel_A：表示物体A的位置和速度，分别为[0, 0, 0]和[0, 0, 0]。

pos_B、vel_B：表示物体B的位置和速度，分别为[1, 0, 0]和[0, 2π, 0]。这里的速度是指围绕圆心旋转的速度，2π表示一周。

pos_C、vel_C：表示物体C的位置和速度，分别为[0.5, √3/2, 0]和[√3π, -0.5π, 0]。这里的速度是指围绕圆心旋转的速度，√3π表示一周。

y0：将上述位置和速度数组合并成一个一维数组，作为三体问题的初始状态向量。

t：通过numpy的linspace函数生成从0到10的5000个等距时间点，用于求解三体问题的解。

In [4]:
pos_A = np.array([0., 0., 0.])
vel_A = np.array([0., 0., 0.])
pos_B = np.array([1., 0., 0.])
vel_B = np.array([0., 2. * np.pi, 0.])
pos_C = np.array([0.5, np.sqrt(3)/2, 0.])
vel_C = np.array([np.sqrt(3)/2 * 2. * np.pi, -0.5 * 2. * np.pi, 0.])

'''Initial condition y0 must be one-dimensional'''
y0 = np.hstack((pos_A, pos_B, pos_C, vel_A, vel_B, vel_C))

t = np.linspace(0, 10, 5000)

# 设置ODE求解器的参数和初始状态。

integrate.ode(f)：

定义了一个名为r的ODE求解器，其中f为上面定义的运动方程函数。

r.set_integrator('dopri5', rtol=1e-12, atol=1e-14)：
设置ODE求解器的参数。这里使用了dopri5求解器，并设置了相对误差容限为1e-12，绝对误差容限为1e-14。


r.set_initial_value(y0, t[0])：

设置ODE求解器的初始状态。y0是三体问题的初始状态向量，t[0]是时间轴的起始时间点。


r.set_f_params(args)：

设置ODE求解器的附加参数。args是一个元组，包含了三体问题的相关参数，包括万有引力常数和三个物体的质量。


In [5]:

r = integrate.ode(f)
r.set_integrator('dopri5', rtol=1e-12, atol=1e-14)
r.set_initial_value(y0, t[0])
r.set_f_params(args)

# 使用ODE求解器求解三体问题的运动方程，并将结果存储在数组中。


dt = t[1] - t[0]：计算时间步长dt，即相邻两个时间点之间的时间差。

y_t = np.zeros((len(t), len(y0)))：初始化一个二维数组y_t，用于存储解。其中，y_t的行数为时间轴的长度，即5000，列数为初始状态向量y0的长度。

idx = 0：初始化一个变量idx，用于记录当前时间点在时间轴上的位置。

while循环的作用：在循环中，判断ODE求解器r是否成功，并且当前时间r.t是否小于时间轴的最后一个时间点。若满足条件，则执行以下操作：

a. y_t[idx, :] = r.y：将ODE求解器当前的状态向量r.y存储在y_t数组的第idx行中，即记录当前时间点的三个物体的位置和速度。

b. r.integrate(r.t + dt)：将ODE求解器的时间进一步推进dt，即计算下一个时间点的状态。

c. idx = idx+1：将idx加1，以记录下一个时间点在y_t数组中的位置。

In [6]:
dt = t[1] - t[0]
y_t = np.zeros((len(t), len(y0)))

idx = 0
while r.successful() and r.t < t[-1]+1e-5:
    y_t[idx, :] = r.y
    r.integrate(r.t + dt)
    idx = idx+1

E:\ML\Anaconda\lib\site-packages\scipy\integrate\_ode.py:1179: UserWarning: dopri5: larger nsteps is needed
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


# 使用bqplot包绘制三体问题的运动轨迹。
figure = plt.figure(title='Bqplot Plot')：创建图形对象figure。

figure.layout.height = '600px'和figure.layout.width = '600px'：设置图形对象figure的高度和宽度为600像素。

plot_A、plot_B和plot_C：分别用红色、蓝色和绿色的线条绘制物体A、B、C在x-y平面上的轨迹。其中，y_t[:, 0]、y_t[:, 3]和y_t[:, 6]表示三个物体在x轴上的位置，y_t[:, 1]、y_t[:, 4]和y_t[:, 7]表示三个物体在y轴上的位置。

scatter_A、scatter_B和scatter_C：分别用红点、蓝点和绿点表示物体A、B、C在初始时间点的位置。其中，y_t[:2, 0]、y_t[:2, 3]和y_t[:2, 6]表示三个物体在x轴上的位置，y_t[:2, 1]、y_t[:2, 4]和y_t[:2, 7]表示三个物体在y轴上的位置。


In [7]:
figure = plt.figure(title='Bqplot Plot')
figure.layout.height = '600px'
figure.layout.width = '600px'

plot_A = plt.plot(y_t[:, 0],y_t[:, 1], 'r')  # A
plot_B = plt.plot(y_t[:, 3],y_t[:, 4], 'b')  # B
plot_C = plt.plot(y_t[:, 6],y_t[:, 7], 'g')  # C
scatter_A = plt.scatter(y_t[:2, 0],y_t[:2, 1], colors=["red"])
scatter_B = plt.scatter(y_t[:2, 3],y_t[:2, 4], colors=["blue"])
scatter_C = plt.scatter(y_t[:2, 6],y_t[:2, 7], colors=["green"])


plt.show()

# 结论：

通过观察轨迹图可以发现，初始时质点的轨道相互靠近，但随着时间的推移，它们的轨道会发生剧烈的变化。